In [1]:
from datetime import datetime

from cali._plate_viewer._analysis_with_sqlmodel import AnalysisRunner
from rich import print

from cali.readers import TensorstoreZarrReader
from cali.sqlmodel import print_experiment_tree, save_experiment_to_database
from cali.sqlmodel._models import AnalysisSettings

# Run from Database Example

In [2]:
from cali.sqlmodel import load_experiment_from_database

In [3]:
database_path = "/Users/fdrgsp/Documents/git/cali/tests/test_data/spontaneous/spont_analysis/cali.db"
exp = load_experiment_from_database(database_path)
print_experiment_tree(exp)

🧪 spontaneous
├── Experiment Type: Spontaneous Activity
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/spontaneous/spont_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)

In [4]:
# delete existing rois
for well in exp.plate.wells:
    for fov in well.fovs:
        fov.rois = []
# save database after deletion
exp = save_experiment_to_database(exp, database_path, overwrite=True)

print_experiment_tree(exp)

🧪 spontaneous
├── Experiment Type: Spontaneous Activity
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/spontaneous/spont_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)

In [7]:
# initialize analysis runner
runner = AnalysisRunner()

# Using the Experiment info, this automatically sets the analysis settings (if any) and
# loads the data (if the experiment has a data path).
#
# If you want to only update the analysis settings, you can use:
# runner.update_settings(analysis_settings).
#
# If you want to to only set the data you can use:
# runner.set_data(data) - (where data is a path, a str or cali reader)

runner.set_experiment(exp)
assert runner.data() is not None

In [ ]:
# set analysis settings (if you dont want to use existing ones)
analysis_settings = AnalysisSettings(
    experiment_id=exp.id,
    created_at=datetime(2025, 11, 8, 20, 25, 1, 911683),
    threads=5,
    neuropil_inner_radius=0,
    neuropil_min_pixels=0,
    neuropil_correction_factor=0.0,
    dff_window=30,
    decay_constant=0.0,
    peaks_height_value=3.0,
    peaks_prominence_multiplier=1.0,
    peaks_height_mode="multiplier",
    peaks_distance=2,
    calcium_sync_jitter_window=5,
    calcium_network_threshold=90.0,
    spikes_sync_cross_corr_lag=5,
    spike_threshold_value=1.0,
    spike_threshold_mode="multiplier",
    burst_threshold=30.0,
    burst_min_duration=3,
    burst_gaussian_sigma=2.0,
)
exp.analysis_settings = analysis_settings

runner.update_settings(analysis_settings)
assert analysis_settings == runner.settings()
print(analysis_settings)

AnalysisSettings(
    created_at=datetime.datetime(2025, 11, 8, 20, 25, 1, 911683),
    neuropil_inner_radius=0,
    neuropil_min_pixels=0,
    neuropil_correction_factor=0.0,
    decay_constant=0.0,
    dff_window=30,
    peaks_height_value=3.0,
    peaks_height_mode='multiplier',
    peaks_distance=2,
    peaks_prominence_multiplier=1.0,
    calcium_sync_jitter_window=5,
    calcium_network_threshold=90.0,
    spike_threshold_value=1.0,
    spike_threshold_mode='multiplier',
    burst_threshold=30.0,
    burst_min_duration=3,
    burst_gaussian_sigma=2.0,
    spikes_sync_cross_corr_lag=5,
    threads=5,
    id=None,
    led_power_equation=None,
    led_pulse_duration=None,
    led_pulse_powers=None,
    led_pulse_on_frames=None,
    stimulation_mask_path=None,
    stimulation_mask_id=None
)

In [13]:
runner.run()

2025-11-10 18:03:37,347 - cali_logger - INFO - Starting Analysis...
2025-11-10 18:03:37,348 - cali_logger - INFO - Starting traces analysis...
2025-11-10 18:03:37,348 - cali_logger - INFO - Number of threads for analysis: 5
2025-11-10 18:03:37,348 - cali_logger - INFO - Positions to analyze: [0]
2025-11-10 18:03:37,374 - cali_logger - INFO - Extracting Traces Data from Well B5_0000_p0.
Extracting Traces Data from Well B5_0000_p0.: 100%|██████████| 4/4 [00:00<00:00, 202.90it/s]
2025-11-10 18:03:37,395 - cali_logger - INFO - Position 0 completed.
2025-11-10 18:03:37,395 - cali_logger - INFO - Analysis Finished.
2025-11-10 18:03:37,396 - cali_logger - INFO - Saving results to database...
2025-11-10 18:03:37,404 - cali_logger - ERROR - An error occurred: (sqlite3.IntegrityError) NOT NULL constraint failed: analysis_settings.experiment_id
[SQL: UPDATE analysis_settings SET experiment_id=? WHERE analysis_settings.id = ?]
[parameters: (None, 1)]
(Background on this error at: https://sqlalche.

In [11]:
print_experiment_tree(exp)

🧪 spontaneous
├── Experiment Type: Spontaneous Activity
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/spontaneous/spont_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)
            ├── 🔬 ROI 1 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 2 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 3 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            └── 🔬 ROI 4 - 🔋 active - ✨ spontaneous
                ├── 📊 Trace data available
                ├── 📈 Data analysis available
                └── 🎭 ROI mask available

# Run From Directories Example

In [ ]:
import useq

from cali.sqlmodel import Experiment, useq_plate_plan_to_db

In [ ]:
database_path = "/Users/fdrgsp/Desktop/cali_test/cali_new.db"
data_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk.tensorstore.zarr"
)
labels_path = "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_labels"
analysis_path = "/Users/fdrgsp/Desktop/cali_test"

new_exp = Experiment(
    name="New Experiment",
    description="A Test Experiment.",
    created_at=datetime.now(),
    database_path=database_path,
    data_path=data_path,
    labels_path=labels_path,
    analysis_path=analysis_path,
)
print(new_exp)

In [ ]:
# load the data and get the useq plate plan from the sequence
data = TensorstoreZarrReader(new_exp.data_path)
plate_plan = data.sequence.stage_positions
assert isinstance(plate_plan, useq.WellPlatePlan)

# Define plate maps for conditions
plate_maps = {
    "genotype": {"B5": "WT"},
    "treatment": {"B5": "Vehicle"},
}

# Create plate with plate_maps and conditions in one step
plate = useq_plate_plan_to_db(plate_plan, new_exp, plate_maps)
new_exp.plate = plate

print(new_exp.plate)
print_experiment_tree(new_exp)

In [ ]:
analysis_settings = AnalysisSettings(
    neuropil_inner_radius=0,
    peaks_prominence_multiplier=1.0,
    spikes_sync_cross_corr_lag=5,
    neuropil_min_pixels=0,
    calcium_sync_jitter_window=5,
    led_power_equation="y = 11 * x - 6",
    neuropil_correction_factor=0.0,
    calcium_network_threshold=90.0,
    led_pulse_duration=100,
    led_pulse_powers=[2.0, 4.0, 6.0],
    led_pulse_on_frames=[3, 53, 103],
    decay_constant=0.0,
    spike_threshold_value=1.0,
    stimulation_mask_path="/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/stimulation_mask.tif",
    id=1,
    dff_window=30,
    spike_threshold_mode="multiplier",
    threads=1,
    peaks_height_value=3.0,
    burst_threshold=30.0,
    experiment_id=1,
    peaks_height_mode="multiplier",
    burst_min_duration=3,
    stimulation_mask_id=1,
    created_at=datetime(2025, 11, 8, 20, 25, 1, 911683),
    peaks_distance=2,
    burst_gaussian_sigma=2.0,
)
new_exp.analysis_settings = analysis_settings

print_experiment_tree(new_exp)

In [ ]:
runner = AnalysisRunner()
runner.set_experiment(new_exp)
data = TensorstoreZarrReader(new_exp.data_path)
runner.set_data(data)

# set which positions to analyze
new_exp.positions_analyzed = list(range(len(plate_plan)))
print(f"Positions to analyze: {new_exp.positions_analyzed}")


def _p(msg: str) -> None:
    print("ANALYSIS INFO:", msg)


runner.analysisInfo.connect(_p)

In [ ]:
runner.run()

In [ ]:
exp_after_analysis = load_experiment_from_database(database_path)
print_experiment_tree(exp_after_analysis)